#### 1. Find the degree of each vertex and store it in a dictionary, sorted by degree.

In [ ]:
def vertex_degrees(graph):
    """
    Calculate degree of each vertex in a graph (adjacency list format) and return sorted by degree.
    
    Args:
        graph: Dictionary representing adjacency list {vertex: [neighbors]}
    
    Returns:
        List of tuples (vertex, degree) sorted by degree in ascending order
    """
    degree_dict = {}
    for vertex in graph:
        degree_dict[vertex] = len(graph[vertex])
    
    # Sort by degree (value) then by vertex (key) for ties
    sorted_degrees = sorted(degree_dict.items(), key=lambda item: (item[1], item[0]))
    return sorted_degrees

# Example usage:
# graph = {'A': ['B', 'C'], 'B': ['A'], 'C': ['A']}
# print(vertex_degrees(graph))  # Output: [('B', 1), ('C', 1), ('A', 2)]

#### 2. Inter-convert the three graph representations (adjacency list, adjacency matrix, and edge list).

In [4]:
def adj_list_to_matrix(adj_list):
    """
    Convert adjacency list to adjacency matrix.
    
    Args:
        adj_list: Dictionary {vertex: [neighbors]}
    
    Returns:
        Tuple of (matrix, vertex_order) where matrix is 2D list and vertex_order maps indices to vertex names
    """
    vertices = sorted(adj_list.keys())
    size = len(vertices)
    matrix = [[0]*size for _ in range(size)]
    
    for i, v in enumerate(vertices):
        for neighbor in adj_list[v]:
            j = vertices.index(neighbor)
            matrix[i][j] = 1
    return matrix, vertices

def adj_matrix_to_list(matrix, vertex_order):
    """
    Convert adjacency matrix to adjacency list.
    
    Args:
        matrix: 2D adjacency matrix
        vertex_order: List mapping matrix indices to vertex names
    
    Returns:
        Adjacency list dictionary
    """
    adj_list = {}
    for i, v in enumerate(vertex_order):
        neighbors = []
        for j in range(len(matrix[i])):
            if matrix[i][j] == 1:
                neighbors.append(vertex_order[j])
        adj_list[v] = neighbors
    return adj_list

def adj_list_to_edge_list(adj_list):
    """
    Convert adjacency list to edge list.
    
    Args:
        adj_list: Dictionary {vertex: [neighbors]}
    
    Returns:
        List of tuples representing edges
    """
    edges = set()  # Using set to avoid duplicates
    for v in adj_list:
        for neighbor in adj_list[v]:
            # Store edge in sorted order to avoid (u,v) and (v,u) duplicates
            edge = tuple(sorted((v, neighbor)))
            edges.add(edge)
    return list(edges)

def edge_list_to_adj_list(edges, directed=False):
    """
    Convert edge list to adjacency list.
    
    Args:
        edges: List of edge tuples
        directed: Boolean indicating if graph is directed
    
    Returns:
        Adjacency list dictionary
    """
    adj_list = {}
    for u, v in edges:
        if u not in adj_list:
            adj_list[u] = []
        if v not in adj_list:
            adj_list[v] = []
            
        adj_list[u].append(v)
        if not directed:
            adj_list[v].append(u)
    return adj_list

#### 3. Check if two vertices are adjacent.


In [3]:
def are_adjacent(graph, u, v):
    """
    Check if two vertices are adjacent in a graph (adjacency list format).
    
    Args:
        graph: Adjacency list dictionary
        u, v: Vertices to check
    
    Returns:
        Boolean indicating adjacency
    """
    # Check both directions since graph could be directed or undirected
    return (v in graph.get(u, [])) or (u in graph.get(v, []))

#### 4. Check if a graph is complete.

In [ ]:
def is_complete(graph):
    """
    Check if a graph (adjacency list) is complete (all possible edges exist).
    
    Args:
        graph: Adjacency list dictionary
    
    Returns:
        Boolean indicating if graph is complete
    """
    vertices = list(graph.keys())
    n = len(vertices)
    
    # A complete graph has n(n-1)/2 edges for undirected
    for v in vertices:
        # All other vertices should be neighbors
        if len(graph[v]) != n - 1 or v in graph[v]:  # No self-loops in complete graph
            return False
    return True

#### 5. Check if a graph is connected.

In [ ]:
def is_connected(graph):
    """
    Check if an undirected graph (adjacency list) is connected using BFS.
    
    Args:
        graph: Adjacency list dictionary
    
    Returns:
        Boolean indicating if graph is connected
    """
    if not graph:
        return True  # Empty graph is trivially connected
        
    visited = set()
    queue = []
    
    # Start with first vertex
    start = next(iter(graph))
    queue.append(start)
    visited.add(start)
    
    while queue:
        v = queue.pop(0)
        for neighbor in graph[v]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return len(visited) == len(graph)

#### 6. Given a graph and a list of vertices, check if it forms a walk, or a trail or a path or none of these.

In [ ]:
def check_sequence_type(graph, vertex_sequence):
    """
    Check if a vertex sequence is a walk, trail, path, or none.
    
    Args:
        graph: Adjacency list
        vertex_sequence: List of vertices
    
    Returns:
        String indicating the type ('walk', 'trail', 'path', or 'none')
    """
    if len(vertex_sequence) < 2:
        return 'none'
    
    # First check if it's a walk (all consecutive vertices adjacent)
    edges_used = set()
    vertices_seen = set()
    is_walk = True
    is_trail = True
    is_path = True
    
    for i in range(len(vertex_sequence)-1):
        u = vertex_sequence[i]
        v = vertex_sequence[i+1]
        
        # Check adjacency
        if v not in graph.get(u, []):
            is_walk = False
            break
            
        # Check for repeated edges (for trail)
        edge = tuple(sorted((u, v)))
        if edge in edges_used:
            is_trail = False
        edges_used.add(edge)
        
        # Check for repeated vertices (for path)
        if i < len(vertex_sequence)-1 and u in vertices_seen:
            is_path = False
        vertices_seen.add(u)
    
    # Check last vertex for path
    if vertex_sequence[-1] in vertices_seen:
        is_path = False
    
    if not is_walk:
        return 'none'
    if is_path:
        return 'path'
    if is_trail:
        return 'trail'
    return 'walk'

#### 7. Check if a given graph is a tree

In [ ]:
def is_tree(graph):
    """
    Check if graph is a tree (connected and acyclic).
    
    Args:
        graph: Adjacency list
    
    Returns:
        Boolean indicating if graph is a tree
    """
    if not graph:
        return False
        
    # Tree must have exactly n-1 edges
    edge_count = sum(len(edges) for edges in graph.values())
    if edge_count != 2 * (len(graph) - 1):  # Each edge counted twice in undirected adj list
        return False
    
    # Tree must be connected
    if not is_connected(graph):
        return False
    
    # No need to explicitly check for cycles since connected graph with n-1 edges is acyclic
    return True

#### 8. Given a connected cyclic graph, find its spanning tree.

In [ ]:
def find_spanning_tree(graph):
    """
    Find a spanning tree using DFS (works for any connected graph).
    
    Args:
        graph: Adjacency list of connected graph
    
    Returns:
        Adjacency list of spanning tree
    """
    if not graph:
        return {}
        
    visited = set()
    spanning_tree = {v: [] for v in graph}
    
    def dfs(v):
        visited.add(v)
        for neighbor in graph[v]:
            if neighbor not in visited:
                spanning_tree[v].append(neighbor)
                spanning_tree[neighbor].append(v)
                dfs(neighbor)
    
    start = next(iter(graph))
    dfs(start)
    return spanning_tree

#### 9. Given a tree, find the number of leaf nodes.

In [ ]:
def count_leaves(tree):
    """
    Count leaf nodes (degree 1) in a tree (adjacency list).
    
    Args:
        tree: Adjacency list of a tree
    
    Returns:
        Integer count of leaf nodes
    """
    return sum(1 for v in tree if len(tree[v]) == 1)

#### 10. Given a tree, check if it's a binary tree.

In [ ]:
def is_binary_tree(tree):
    """
    Check if tree is binary (each node has at most 2 children).
    
    Args:
        tree: Adjacency list
    
    Returns:
        Boolean indicating if tree is binary
    """
    # Root has at most 2 children, others have at most 1 parent + 2 children = 3 edges
    for v in tree:
        if len(tree[v]) > 3:
            return False
        # For undirected tree, root has <= 2, others <= 3
        # For directed tree, would need different logic
    return True

#### 11. Given a tree and a node, find its height.

In [ ]:
def node_height(tree, node):
    """
    Find height of a node in a tree (longest path from node to leaf).
    
    Args:
        tree: Adjacency list
        node: Starting node
    
    Returns:
        Integer height
    """
    visited = set()
    
    def height_dfs(v):
        visited.add(v)
        max_h = 0
        for neighbor in tree[v]:
            if neighbor not in visited:
                current_h = height_dfs(neighbor)
                if current_h > max_h:
                    max_h = current_h
        return max_h + 1
    
    return height_dfs(node) - 1  # Subtract 1 to count edges not nodes

#### 12. Given a tree and a node, find its depth.

In [ ]:
def node_depth(tree, root, target):
    """
    Find depth of a node in a tree (path length from root to node).
    
    Args:
        tree: Adjacency list
        root: Root node of tree
        target: Node whose depth to find
    
    Returns:
        Integer depth, or -1 if node not found
    """
    visited = set()
    
    def depth_dfs(v, current_depth):
        if v == target:
            return current_depth
        visited.add(v)
        for neighbor in tree[v]:
            if neighbor not in visited:
                result = depth_dfs(neighbor, current_depth + 1)
                if result != -1:
                    return result
        return -1
    
    return depth_dfs(root, 0)